In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib import rcParams
from matplotlib.legend import Legend
import seaborn as sns
rcParams["font.family"] = "sans-serif"
rcParams['font.sans-serif'] = ['DejaVu Sans']

from astropy.coordinates import SkyCoord
from astropy import units as u

import glob

%matplotlib notebook

## Test a couple files first

In [3]:
def get_hist_counts(df):

    rf_arr = np.array(df.rf_score)

    coords = SkyCoord(np.array(df.raStack)*u.deg, np.array(df.decStack)*u.deg)

    all_cnts, _ = np.histogram(rf_arr, range=(0,1), bins=100)
    low_lat = np.where(np.abs(coords.galactic.b) < 15*u.deg)
    high_lat = np.where(np.abs(coords.galactic.b) > 50*u.deg)

    low_cnts, _ = np.histogram(rf_arr[low_lat], range=(0,1), bins=100)
    high_cnts, _ = np.histogram(rf_arr[high_lat], range=(0,1), bins=100)
    
    return all_cnts, low_cnts, high_cnts

In [4]:
file_list = glob.glob("/Users/adamamiller/Desktop/PS1_fits/hdf5/*h5")

In [6]:
all_cnts = np.zeros(100, dtype=np.int64)
low_cnts = np.zeros_like(all_cnts)
high_cnts = np.zeros_like(all_cnts)

for file in file_list:
    df = pd.read_hdf(file)
    all_tmp, low_tmp, high_tmp = get_hist_counts(df)
    all_cnts += all_tmp
    low_cnts += low_tmp
    high_cnts += high_tmp

In [ ]:
col_dict = {'mustard': "#E29930",
            'blue': "#217CA3",
            'asphalt': "#32384D"
           }

x_grid = np.repeat(np.linspace(0.0, 1, 101), 2)

fig, ax = plt.subplots()
ax.plot(x_grid, np.concatenate(([0], np.repeat(all_cnts, 2), [0])),
        color=col_dict['blue'])
ax.fill(x_grid, np.concatenate(([0], np.repeat(all_cnts, 2), [0])),
        alpha=0.4, color=col_dict['blue'])
ax.plot(x_grid, np.concatenate(([0], np.repeat(low_cnts, 2), [0])), 
        color=col_dict['asphalt'])
ax.fill(x_grid, np.concatenate(([0], np.repeat(low_cnts, 2), [0])),
        alpha=0.4, color=col_dict['asphalt'])
ax.plot(x_grid, np.concatenate(([0], np.repeat(high_cnts, 2), [0])), 
        color=col_dict['mustard'])
ax.fill(x_grid, np.concatenate(([0], np.repeat(high_cnts, 2), [0])),
        alpha=0.4, color=col_dict['mustard'])

# ax.bar(np.linspace(0.005, .995, 100), low_cnts, width=0.01, lw=2, facecolor="None")
# ax.bar(np.linspace(0.005, .995, 100), high_cnts, width=0.01)
ax.set_yscale("log")
ax.set_xlim(-0.01, 1.01)
ax.tick_params(which="both", top=True, right=True, labelsize=11)
ax.yaxis.set_minor_locator(MultipleLocator(0.05))
ax.set_xlabel("RF score")
ax.set_ylabel("N")

fig.subplots_adjust(left=0.1,right=0.99,top=0.98,bottom=0.1)